## Analysis of Simulation Results

ABM in which agents walk over a field to collect and eat sugar. Based on [the ABM Sugarscape 3 Wealth Distribution](https://ccl.northwestern.edu/netlogo/models/Sugarscape3WealthDistribution).

Features:
- Heterogeneous ants: some explore with a wider radius then others (high vs low senses).
- The amount of food (sugar) is has higher concentrations in some areas of the map.

No real ants were hurt in the experiments.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

In [6]:
df=pd.read_csv('ants.csv')
df.head()

,Unnamed: 0,RunId,iteration,Step,initial_population,recreate,internet,solidarity,individualist_percent,Living Ants,Dead Ants,% Dead Low Senses,% Dead High Senses,% Dead Individualist,min_sugar,max_sugar,avg_sugar,stdev_sugar
0,0,1,0,1000,100,0,False,False,0,62,38,0.657895,0.342105,0.0,139.0,2020.0,1381.209677,481.042009
1,1,3,0,1000,100,0,False,False,0,68,32,0.562500,0.437500,0.0,246.0,2012.0,1285.147059,538.134874
2,2,4,0,1000,100,0,False,False,0,68,32,0.625000,0.375000,0.0,70.0,2016.0,1205.691176,565.457053
3,3,2,0,1000,100,0,False,False,0,67,33,0.515152,0.484848,0.0,115.0,2015.0,1362.343284,562.748304
4,4,0,0,1000,100,0,False,False,0,70,30,0.666667,0.333333,0.0,176.0,2015.0,1223.114286,577.336451


In [7]:
len(df)

400

In [84]:
categories = ['Living Ants','Dead Ants','% Dead Low Senses','% Dead High Senses','% Dead Individualist', "min_sugar", "max_sugar", "avg_sugar"]

def normalize_columns(df):
    cut_df = df[categories]
    a = cut_df[categories[0:2]]
    b = cut_df[categories[2:5]]
    c = cut_df[categories[5:]]
    an = (a-a.min())/(a.max()-a.min())
    cn = (c-c.min())/(c.max()-c.min())
    return pd.concat([an,b,cn], axis=1)

### Internet
The internet option adds knowledge about the resource distribution for everyone.

In [94]:
no_internet = df[(df['internet'] == False) & (df['solidarity'] == False) & (df['individualist_percent'] == 0)]
internet = df[(df['internet'] == True) & (df['solidarity'] == False) & (df['individualist_percent'] == 0)]

no_internet_normalized = normalize_columns(no_internet)
internet_normalized = normalize_columns(internet)

In [93]:
fig = go.Figure()
fig.add_trace(go.Scatterpolar(
      r=[no_internet_normalized[c].mean() for c in categories],
      theta=categories,
      name='Base'
))
fig.add_trace(go.Scatterpolar(
      r=[internet_normalized[c].mean() for c in categories],
      theta=categories,
      name='Internet'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 0.7]
    )),
  showlegend=True
)

fig.show()

#### Findings

Adding internet leads to slightly more living agents. However, this further adds to the disadvantage of the low senses agents. Their search and movement radius is smaller compared to the one of the high senses agents. Adding internet increases the percentage of dead low senses agents from 57% to 64%.

### Solidarity
The solidarity option adds a distribution hub to which everyone sends extra food and from where everyone can take food when in need. Transport cost was ignored.

In [95]:
no_solidarity = df[(df['solidarity'] == False) & (df['individualist_percent'] == 0)]
solidarity = df[(df['solidarity'] == True) & (df['individualist_percent'] == 0)]

no_solidarity_normalized = normalize_columns(no_solidarity)
solidarity_normalized = normalize_columns(solidarity)

In [97]:
fig = go.Figure()
fig.add_trace(go.Scatterpolar(
      r=[no_solidarity_normalized[c].mean() for c in categories],
      theta=categories,
      name='Base'
))
fig.add_trace(go.Scatterpolar(
      r=[solidarity_normalized[c].mean() for c in categories],
      theta=categories,
      name='Solidarity'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=True
)

fig.show()

#### Findings

Adding solidarity distribution increases the number of living ants and decreases the disadvantage of low versus high senses ants.

### Individualists
This checks if and at which percentage agents not contributing to the solidarity principle have an impact on the system.

In [106]:
percentages_individualists=[0, 1, 10, 50, 90]

impact_per_percentage = {
    percentage: df[(df['solidarity'] == True) & (df['individualist_percent'] == percentage)]
    for percentage in percentages_individualists
}

impact_per_percentage_normalized = {
    percentage: normalize_columns(impact) 
    for percentage, impact in impact_per_percentage.items()
}
impact_per_percentage[0]["max_sugar"]

7      24.0
17     24.0
25     24.0
31     24.0
45     24.0
50     24.0
61     24.0
79     24.0
81     24.0
92     23.0
103    24.0
110    24.0
123    24.0
132    24.0
141    24.0
159    24.0
160    24.0
172    24.0
183    24.0
190    24.0
203    24.0
211    24.0
221    24.0
239    24.0
240    24.0
253    24.0
264    24.0
270    24.0
283    24.0
291    24.0
301    24.0
319    24.0
320    24.0
331    24.0
343    24.0
350    24.0
363    24.0
372    24.0
381    24.0
394    24.0
Name: max_sugar, dtype: float64

In [102]:
fig = go.Figure()

for percentage in percentages_individualists:
    fig.add_trace(go.Scatterpolar(
          r=[impact_per_percentage_normalized[percentage][c].mean() for c in categories],
          theta=categories,
          name=f"{percentage}% individualists"
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=True
)

fig.show()

#### Findings

Individualists are at higher death risk. E.g. with 50% individualists, 75% of all dead agents were individualists. 

TODO the difference between 0% and 1% is strange. Need to check that again.